In [1]:
%env SM_FRAMEWORK=tf.keras
import zipfile, os, numpy as np, pickle, yaml, gc
nii_size = 384
datatype='3.0T'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

env: SM_FRAMEWORK=tf.keras


In [2]:
np.load('../my_data/datasets/all_seg_data_NL_zm_2/T3_image_arr_384_train.npy').shape

(140, 32, 384, 384)

In [2]:
img_layer_path = '/ssd1/dataset_stack/MRI dataset/is0001-is0350--original_itk--image+mask/'

In [3]:
test_data_name = [i.replace('/ssd1/dataset_stack//MRI dataset//isXXXX_all_mask_and_image_check/3.0T_test/','') for i in np.load('../nii_save/test_data_NL/T3_image_mask_path_valid.npy')]
# print(test_data_name)

In [14]:
import pandas as pd
import nibabel as nib
from scipy import ndimage
mri_content = []
def load_nii(path):
    image_o = nib.load(path)
    if len(image_o.shape) == 4:
            image = image_o.get_fdata()
            width,height,queue,_ = image.shape
            image = image[:,:,:,1]
            image = np.reshape(image,(width,height,queue))
    else:
        image = image_o.get_fdata()
    return image
for next_ in test_data_name:
    image_msg = load_nii(os.path.join(img_layer_path, next_))
    mask_msg = load_nii(os.path.join(img_layer_path, next_.replace('o','s')))
    # print(next_)
    # print('------Image------')
    # print('Image Intensity', "MAX", np.max(image_msg), "MIN", np.min(image_msg), "MEAN", np.mean(image_msg))
    # print('------Masks------')
    # print('Pixel', "SUM", np.sum(mask_msg),)
    mri_content.append([next_[0:-8],np.max(image_msg), np.min(image_msg), np.mean(image_msg), int(np.sum(mask_msg))] )
    

In [16]:
save_information = pd.DataFrame(mri_content, columns=['Test MRI','Max Image Intensity', 'Min Image Intensity', 'Mean Image Intensity', 'Mask Pixel Sum'])
save_information.to_csv('Only Test data.csv', index=False)